In [ ]:
!pip install langchain
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.8 MB/s eta 0:00:00


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ''

In [ ]:
import os
import json
import zipfile
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_openai import ChatOpenAI

# LangChain 모델 설정
chat = ChatOpenAI(model_name="gpt-4o-mini", temperature=1, streaming=False)

# 경로 설정
zip_path = "/content/merged_output.zip"
extract_folder = "/content/news_extracted"
output_folder = "/content/fake_news"
fake_zip_path = "/content/TEST_fake_news.zip"

# 압축 해제
os.makedirs(extract_folder, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_folder)

# 병렬 처리 함수 정의
def process_file(filename):
    if not filename.endswith(".json"):
        return None

    file_path = os.path.join(extract_folder, filename)
    with open(file_path, "r", encoding="utf-8") as file:
        news_data = json.load(file)

    original_title = news_data["sourceDataInfo"]["newsTitle"]
    original_content = news_data["sourceDataInfo"]["newsContent"]

    # 가짜 제목 생성
    fake_title_prompt = f"""
    당신은 뉴스 제목을 재가공하는 역할을 맡았습니다.
    다음은 실제 뉴스 제목입니다: {original_title}

    해당 기사를 다음과 같은 규칙으로 변형하세요:
    - 제목을 다르게 구성하되, 주요 키워드(인물명, 기관명 등)는 포함
    - 의미를 왜곡하거나, 일부 정보(숫자, 인명, 지명 등)를 삭제하거나 변경
    - 제목을 더 모호하거나 애매하게 만들어 독자가 클릭하도록 유도

    아래 사항을 엄수하세요:
    - 출력은 반드시 재작성된 제목만 포함
    - 부가적인 설명, 접두사(예: "변형된 제목:"), 따옴표 등은 일절 금지
    - 결과는 단독 문자열만 출력할 것
    """
    fake_title = chat.predict(fake_title_prompt)

    # 가짜 본문 생성
    fake_content_prompt = f"""
    당신은 뉴스 본문을 변형하는 역할을 맡았습니다.
    다음은 실제 뉴스 내용입니다: {original_content}

    해당 기사를 다음과 같은 규칙으로 변형하세요:
    - 본문에서 일부 정보(숫자, 인명, 지명 등)를 삭제하거나 맥락과 맞지 않는 정보로 변경
    - 일부 문장은 비정상적으로 끝나게 수정
    - 내용의 전반적인 주제는 유지하지만, 세부 정보가 틀리도록 수정
    - 원본 기사보다 감정적 표현(분노, 두려움, 흥분 등)을 추가
    - 객관적 사실 대신, 주관적 해석이 포함된 표현 추가
    - 전문가 의견이나 연구 결과를 과장 또는 왜곡하여 삽입

    아래 사항을 엄수하세요:
    - 출력은 반드시 재작성된 본문만 포함
    - 부가적인 설명, 접두사(예: "변형된 내용:"), 따옴표 등은 일절 금지
    - 결과는 뉴스 본문처럼 보이도록 단독 텍스트만 출력하세요
    """
    fake_news = chat.predict(fake_content_prompt)

    # 결과 저장
    fake_news_data = {
        "sourceDataInfo": {
            "newsTitle": fake_title,
            "newsContent": fake_news,
            "newsCompany": news_data["sourceDataInfo"].get("newsCompany", "Unknown"),
            "newsDate": news_data["sourceDataInfo"].get("newsDate", "Unknown"),
            "newsCategory": news_data["sourceDataInfo"].get("newsCategory", "Unknown"),
            "newsReporter": news_data["sourceDataInfo"].get("newsReporter", "Unknown")
        }
    }

    fake_file_path = os.path.join(output_folder, f"fake_{filename}")
    with open(fake_file_path, "w", encoding="utf-8") as fake_file:
        json.dump(fake_news_data, fake_file, ensure_ascii=False, indent=4)

    return fake_file_path

# 병렬 실행
fake_news_files = []
with ThreadPoolExecutor(max_workers=5) as executor:  # max_workers 조절 가능
    futures = [executor.submit(process_file, fname) for fname in os.listdir(extract_folder)]
    for future in as_completed(futures):
        result = future.result()
        if result:
            fake_news_files.append(result)

# ZIP 파일로 저장
with zipfile.ZipFile(fake_zip_path, "w") as zipf:
    for file in fake_news_files:
        zipf.write(file, os.path.basename(file))

print(f"가짜 뉴스 ZIP 파일 저장 완료: {fake_zip_path}")

가짜 뉴스 ZIP 파일 저장 완료: /content/ED_fake_news.zip
